In [ ]:
!pip install ultralytics -q
import torch
from ultralytics import YOLO

# Verify GPU
print(f"Using Device: {torch.cuda.get_device_name(0)}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Using Device: Tesla T4


In [ ]:
import cv2
import numpy as np
import os
import random

IMG_SIZE = 640
TRAIN_COUNT = 3000
VAL_COUNT = 600

SHAPES = ["circle", "triangle", "square"]

# --- HSV COLOR RANGES (CRITICAL FIX) ---
HSV_RANGES = {
    "red":   [(0, 10), (170, 180)],
    "green": [(40, 85)],
    "blue":  [(95, 135)]
}

# --- CLASS MAP (MUST MATCH YAML EXACTLY) ---
CLASS_NAMES = []
CLASS_MAP = {}
idx = 0
for shape in SHAPES:
    for color in ["red", "green", "blue"]:
        CLASS_MAP[(shape, color)] = idx
        CLASS_NAMES.append(f"{color}_{shape}")
        idx += 1


# ---------- UTILITIES ----------

def hsv_color(color_name):
    ranges = HSV_RANGES[color_name]
    h_range = random.choice(ranges)
    h = random.randint(h_range[0], h_range[1])
    s = random.randint(140, 255)
    v = random.randint(140, 255)
    hsv = np.uint8([[[h, s, v]]])
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)[0][0].tolist()


def random_background():
    base = random.randint(150, 230)
    bg = np.ones((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8) * base
    noise = np.random.normal(0, 12, bg.shape).astype(np.int16)
    bg = np.clip(bg.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    return bg


def photometric_distort(img):
    alpha = random.uniform(0.7, 1.3)   # contrast
    beta = random.randint(-25, 25)     # brightness
    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    if random.random() < 0.4:
        img = cv2.GaussianBlur(img, (3, 3), 0)
    return img


def draw_shape(img, shape, color_bgr):
    size = random.randint(120, 260)
    cx = random.randint(size, IMG_SIZE - size)
    cy = random.randint(size, IMG_SIZE - size)

    if shape == "circle":
        r = size // 2
        cv2.circle(img, (cx, cy), r, color_bgr, -1)
        bbox = (cx - r, cy - r, cx + r, cy + r)

    elif shape == "square":
        jitter = random.randint(-15, 15)
        x1 = cx - size//2 + jitter
        y1 = cy - size//2 + jitter
        x2 = cx + size//2 + jitter
        y2 = cy + size//2 + jitter
        cv2.rectangle(img, (x1,y1), (x2,y2), color_bgr, -1)
        bbox = (x1, y1, x2, y2)

    elif shape == "triangle":
        pts = np.array([
            [cx + random.randint(-20,20), cy - size//2],
            [cx - size//2, cy + size//2],
            [cx + size//2, cy + size//2],
        ])
        cv2.fillPoly(img, [pts], color_bgr)
        x, y, w, h = cv2.boundingRect(pts)
        bbox = (x, y, x + w, y + h)

    return bbox


def yolo_bbox(b):
    x1, y1, x2, y2 = b
    xc = ((x1 + x2) / 2) / IMG_SIZE
    yc = ((y1 + y2) / 2) / IMG_SIZE
    w  = (x2 - x1) / IMG_SIZE
    h  = (y2 - y1) / IMG_SIZE
    return xc, yc, w, h


# ---------- GENERATION ----------

def generate(split, count):
    img_dir = f"dataset1/images/{split}"
    lbl_dir = f"dataset1/labels/{split}"
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(lbl_dir, exist_ok=True)

    for i in range(count):
        img = random_background()
        labels = []

        # --- MULTI OBJECT (IMPORTANT) ---
        obj_count = random.choices([1,2,3], weights=[0.6,0.3,0.1])[0]

        for _ in range(obj_count):
            shape = random.choice(SHAPES)
            color_name = random.choice(["red","green","blue"])
            color = hsv_color(color_name)

            bbox = draw_shape(img, shape, color)
            cls = CLASS_MAP[(shape, color_name)]
            xc,yc,w,h = yolo_bbox(bbox)
            labels.append(f"{cls} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

        img = photometric_distort(img)

        cv2.imwrite(f"{img_dir}/{i}.jpg", img)
        with open(f"{lbl_dir}/{i}.txt", "w") as f:
            f.write("\n".join(labels))


# ---------- RUN ----------
generate("train", TRAIN_COUNT)
generate("val", VAL_COUNT)



In [ ]:
# 1. Load the model (using pretrained weights to speed up convergence)
model = YOLO('yolov8n.pt')

# 2. Start Professional Training
results = model.train(
    data='/content/dataset1/data.yaml',      # Path to your yaml file
    epochs=50,            # Max epochs (Early stopping will likely hit sooner)
    imgsz=640,             # Your generated image size
    batch=32,              # High batch size for T4 GPU efficiency
    device=0,              # Use the T4 GPU
    patience=10,           # STOP training if no improvement for 10 epochs (No Overfitting)
    save=True,             # Save checkpoints
    exist_ok=True,         # Overwrite existing runs for clean workspace
    optimizer='AdamW',     # Professional standard for stable convergence
    lr0=0.001,             # Initial learning rate
    augment=True,          # Enable internal augmentations for better generalization
    plots=True
    )

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, 

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a17a336e3e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 30% ━━━╸──────── 3/10 1.1s/it 2.2s<8.0s

^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a17a336e3e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 60% ━━━━━━━───── 6/10 1.1it/s 4.9s<3.8s

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a17a336e3e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a17a336e3e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 16

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 70% ━━━━━━━━──── 7/10 1.1s/it 6.7s<3.3s

can only test a child process


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 1.0it/s 9.7s
                   all        600        900      0.984      0.964      0.988      0.964

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      11/50      4.97G     0.3067     0.3952     0.9074         66        640: 100% ━━━━━━━━━━━━ 94/94 1.8it/s 51.5s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 1.6it/s 6.4s
                   all        600        900      0.968      0.977      0.988      0.963

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      12/50      4.97G     0.3068     0.3903      0.908         66        640: 100% ━━━━━━━━━━━━ 94/94 1.9it/s 49.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 1.4it/s 6.9s
                   all        600        900     

KeyboardInterrupt: 

In [ ]:
# Colab script: YOLO validation + VIDEO inference (SAVE FULL ANNOTATED VIDEO)

import cv2
import os
from ultralytics import YOLO

def evaluate_and_predict_video_save(
    model_path,
    video_path,
    output_path,
    conf=0.5
):
    # 1. Load model
    model = YOLO(model_path)

    # 2. Run Validation
    print("\n--- Starting Validation ---")
    metrics = model.val()

    # 3. Log KPIs
    print("\n--- Key Performance Indicators ---")
    print(f"mAP50:       {metrics.box.map50:.3f}")
    print(f"mAP50-95:    {metrics.box.map.mean():.3f}")
    print(f"Precision:   {metrics.box.mp:.3f}")
    print(f"Recall:      {metrics.box.mr:.3f}")

    # 4. Open video
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Cannot open input video"

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)

    # 5. Video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_idx = 0

    print("\n--- Running VIDEO Inference & Saving ---")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(
            source=frame,
            conf=conf,
            verbose=False
        )

        annotated = results[0].plot()
        out.write(annotated)

        if frame_idx % 30 == 0:
            print(f"Processed frame {frame_idx}")

        frame_idx += 1

    cap.release()
    out.release()

    print(f"\n--- Done ---")
    print(f"Annotated video saved to: {output_path}")


# --- EXECUTION ---
MODEL_WTS = "/content/runs/detect/train/weights/best.pt"
INPUT_VIDEO = "/content/test-video.mp4"        # upload this
OUTPUT_VIDEO = "/content/output_annotated.mp4"

evaluate_and_predict_video_save(
    model_path=MODEL_WTS,
    video_path=INPUT_VIDEO,
    output_path=OUTPUT_VIDEO,
    conf=0.5
)



--- Starting Validation ---
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2861.9±552.8 MB/s, size: 147.4 KB)
val: Scanning /content/dataset1/labels/val.cache... 600 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 600/600 917.1Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 38/38 4.8it/s 8.0s
                   all        600        900      0.991      0.974       0.99      0.976
            red_circle         93         96      0.994       0.99      0.994      0.979
          green_circle         99        105          1      0.997      0.995      0.984
           blue_circle         88         92       0.99      0.957      0.986      0.971
          red_triangle         92         96          1      0.959       0.98      0.972
        green_trian